In [4]:
#Import Data Handling Packages
import numpy as np
import pandas as pd
from datetime import datetime
import itertools as it
import os
import codecs
import warnings

#Import Natural Language Packages
import spacy
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim
from sklearn.manifold import TSNE


#Some Other Stuff
%matplotlib inline
nlp = spacy.load('en')

In [5]:
#Reading the appropriate CSV's
Groupons = pd.read_csv('~/CodingProjects/WebScrap/groupon/groupon_deals.csv')
reviews1 = pd.read_csv('~/CodingProjects/WebScrap/groupon_reviews/groupon_reviews_1.csv')
reviews2 = pd.read_csv('~/CodingProjects/WebScrap/groupon_reviews/groupon_reviews_2.csv')
reviews3 = pd.read_csv('~/CodingProjects/WebScrap/groupon_reviews/groupon_reviews_3.csv')
reviews4 = pd.read_csv('~/CodingProjects/WebScrap/groupon_reviews/groupon_reviews_4.csv')
reviews5 = pd.read_csv('~/CodingProjects/WebScrap/groupon_reviews/groupon_reviews_5.csv')
reviews6 = pd.read_csv('~/CodingProjects/WebScrap/groupon_reviews/groupon_reviews_6.csv')

In [6]:
#Convert the Ratings column -> just numbers and change type to int
Groupons['total_ratings'] = [i.replace(' Ratings','').replace(',','') for i in Groupons['total_ratings']]
Groupons['total_ratings'] = Groupons['total_ratings'].apply(pd.to_numeric)

In [7]:
#Convert the categories column to a nested list and remove the 'Local' tag present in all
fix_categ_1 = [i.split(',')[1:] for i in Groupons['categories']]
Groupons['categories'] = fix_categ_1
#Fixes the one mistake in tags
Groupons.iloc[9,0][0] = 'Health & Fitness'

In [8]:
#Convert the mini-info column to a nested list
fix_num_1 = [j for j in [i.split(',') for i in Groupons['mini_info']] if j != '']
Groupons['mini_info'] = fix_num_1

In [9]:
#Create a mega pandas column of all individual reviews
all_groupon_reviews = pd.concat([reviews1,reviews2,reviews3,reviews4,reviews5,reviews6], ignore_index=True)

In [10]:
#Remove the word 'Redeemed' from date column
all_groupon_reviews['date'] = [i.replace('Redeemed','').strip() for i in all_groupon_reviews['date']]

#Convert the date column to actual date objects
#First define a function to apply
def to_date(x):
    try:
        return datetime.strptime(x, '%B %d, %Y')
    except:
        return datetime.strptime('October 19, 2017', '%B %d, %Y')

all_groupon_reviews['date'] = all_groupon_reviews['date'].apply(to_date)

#Add Three Columns -> year, month, day
all_groupon_reviews['year'] = [i.year for i in all_groupon_reviews.date]
all_groupon_reviews['month'] = [i.month for i in all_groupon_reviews.date]
all_groupon_reviews['day'] = [i.day for i in all_groupon_reviews.date]

In [11]:
#####################################
# Category column -> Just 1 category#
#####################################
# all_groupon_reviews.head()
# Groupons.head(10)
#There seems to be a brewing company [row ==14] that doesn't have any more tags

# x= {}
# for i in Groupons.categories:
#     try:
#         x.update({i[0]:set()})
#     except:
#         print('Warning: ', i[0])
# count = 0   
# for i in Groupons.categories:
#     try:
#         x[i[0]].add(i[1])
#     except:
#         print('Warning: No index at: ', i, count)
#     count += 1 
# x
# filter_indxes = []
# for idx, j in enumerate(Groupons.categories):
#     try:
#         if j[1] == 'Auto Repair & Maintenance':
#             filter_indxes.append(idx)
#         else:
#             continue
#     except:
#         continue
# Groupons.iloc[filter_indxes,:].shape
for idx, content in enumerate(Groupons.categories):
    Groupons.iloc[idx,0] = content[0]

In [12]:
###################################################################################
# Just a neat representation of how many in each category and their total ratings #
###################################################################################
Groupons.groupby('categories').agg({'categories': 'count', 'total_ratings':['min', 'mean', 'max']})

categories total_ratings                      
                       count           min          mean     max
categories                                                      
Automotive                 9            28   8395.666667   48838
Beauty & Spas            173             3    877.323699   24088
Food & Drink             118             1   1246.983051   49414
Health & Fitness          45             5    735.755556   20320
Home Services              1            54     54.000000      54
Personal Services         15             9  34187.733333  181381
Things To Do              28           134   3359.178571   45249

In [ ]:
auto_url_list = list(Groupons.loc[Groupons.loc[:,'categories']=='Automotive',:].url)
things_url_list = list(Groupons.loc[Groupons.loc[:,'categories']=='Things To Do',:].url)
home_url_list = list(Groupons.loc[Groupons.loc[:,'categories']=='Home Services',:].url)
personal_url_list = list(Groupons.loc[Groupons.loc[:,'categories']=='Personal Services',:].url)
health_url_list = list(Groupons.loc[Groupons.loc[:,'categories']=='Health & Fitness',:].url)
food_url_list = list(Groupons.loc[Groupons.loc[:,'categories']=='Food & Drink',:].url)
beauty_url_list = list(Groupons.loc[Groupons.loc[:,'categories']=='Beauty & Spas',:].url)

In [ ]:
auto_reviews = all_groupon_reviews[all_groupon_reviews['url'].isin(auto_url_list)]
things_reviews = all_groupon_reviews[all_groupon_reviews['url'].isin(things_url_list)]
home_reviews = all_groupon_reviews[all_groupon_reviews['url'].isin(home_url_list)]
personal_reviews = all_groupon_reviews[all_groupon_reviews['url'].isin(personal_url_list)]
health_reviews = all_groupon_reviews[all_groupon_reviews['url'].isin(health_url_list)]
food_reviews = all_groupon_reviews[all_groupon_reviews['url'].isin(food_url_list)]
beauty_reviews = all_groupon_reviews[all_groupon_reviews['url'].isin(beauty_url_list)]

In [ ]:
print(auto_reviews.shape)
print(things_reviews.shape)
print(home_reviews.shape)
print(personal_reviews.shape)
print(health_reviews.shape)
print(food_reviews.shape)
print(beauty_reviews.shape)
all_groupon_reviews.head()

In [13]:
#intermediate_directory = os.path.join("C:",".",'intermediate')
review_txt_filepath = os.path.join(os.getcwd(), 'review_text_all.txt')

In [14]:
print(all_groupon_reviews[all_groupon_reviews.content.apply(lambda x: isinstance(x, float))])
indx = [10096]
all_groupon_reviews.content.iloc[indx]


            author       date content  \
10096  Patricia D. 2017-02-15     NaN   
15846       Pat H. 2016-09-24     NaN   
19595      Tova F. 2012-12-20     NaN   
40328   Phyllis H. 2015-06-28     NaN   
80140     Andre A. 2013-03-26     NaN   

                                                 url  year  month  day  
10096  https://www.groupon.com/deals/statler-grill-9  2017      2   15  
15846         https://www.groupon.com/deals/impark-3  2016      9   24  
19595   https://www.groupon.com/deals/hair-bar-nyc-1  2012     12   20  
40328     https://www.groupon.com/deals/kumo-sushi-1  2015      6   28  
80140  https://www.groupon.com/deals/woodburybus-com  2013      3   26  


10096    NaN
Name: content, dtype: object

In [15]:
review_count = 0
with codecs.open(review_txt_filepath, 'w', encoding = 'utf_8') as review_txt_file:
    for i in all_groupon_reviews.content.dropna():
        review_txt_file.write(i.replace('\n', '\\n') + '\n')
        review_count += 1
print('Text from {:,} reviews written to the new txt file'.format(review_count))

Text from 88,560 reviews written to the new txt file


In [16]:
def punct_space(token):
    return token.is_punct or token.is_space
def line_review(filename):
    with codecs.open(filename, encoding = 'utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
def lemmatized_sentence_corpus(filename):
    for parsed_review in nlp.pipe(line_review(filename), batch_size=10000, n_threads=3):
        for sent in parsed_review.sents:
            yield ' '.join([token.lemma_ for token in sent if not punct_space(token)])


In [17]:
unigram_sentences_filepath = os.path.join(os.getcwd(), 'unigram_sentences_all.txt')

In [18]:
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')

In [19]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [21]:
# for unigram_sentence in unigram_sentences:
#     print(' '.join(unigram_sentence))
#     print('')

In [20]:
bigram_model_filepath = os.path.join(os.getcwd(), 'bigram_model_all')

In [22]:
%%time
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)

CPU times: user 3.38 s, sys: 81.3 ms, total: 3.46 s
Wall time: 3.5 s


In [23]:
bigram_model = Phrases.load(bigram_model_filepath)

In [24]:
bigram_sentences_filepath = os.path.join(os.getcwd(), 'bigram_sentences_all.txt')

In [25]:
%%time
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = ' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

/Users/nicholasmaloof/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 7.62 s, sys: 87.6 ms, total: 7.71 s
Wall time: 7.78 s


In [26]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [27]:
for bigram_sentence in bigram_sentences:
    print(' '.join(bigram_sentence))

joseph have a wonderful disposition
-PRON- be put at_ease immediately
-PRON- ask question and listen
-PRON- feel -PRON- be genuinely concerned_about address -PRON- issue
the place be beautifully serene neatly design immaculate but certainly not sterile or detached
-PRON- be so relaxed -PRON- think -PRON- snore for a second
-PRON- feel great
-PRON- will be return for sure
brian do an excellent job address -PRON- issue
after the procedure -PRON- feel uplift and the pain be go
amazing service
be -PRON- first time get acupuncture and brian help -PRON- feel so comfortable
after the appointment -PRON- feel incredible
-PRON- body be hum and -PRON- be totally relaxed
will be go back for more appointment
-PRON- have no clue what to expect with acupuncture
-PRON- just seem like something -PRON- would try one day to see what -PRON- be like
-PRON- will l love -PRON-
-PRON- love the experience at city acupuncture-- great temperature low light quite sound low peaceful music calm gentle and comfortin

keep up the great work
excellent thorough service
have a great experience
devon and lisa be very pleasant and professional
-PRON- be plan on take -PRON- car for an oil_change at firestone where -PRON- would have spend 3 hour for an oil_change
-PRON- love -PRON- where -PRON- do not have to leave -PRON- car in order to get service and only take 20_minute
-PRON- will frequent valvoline in the piscataway location
great service
the service be very good
the employee be very helpful
quick helpful experience
the staff at valvoline in watertown be fantastic
very connected to -PRON- customer great customer service
with the groupon -PRON- be a great experience all around
great service
angelica be really sweet and so be everyone else
-PRON- be in and out and be back to work before -PRON- lunch_break be over
very professional quick fast and a great job
-PRON- go to the lemoyne location
the gentleman_who assist -PRON- be informative and professional
-PRON- explain -PRON- be go on a trip and -PRON- t

perfect work
sell thing not needed and -PRON- could have saved more $ $ go to -PRON- mercedes dealer
-PRON- would never charge -PRON- for extra
always very happy with -PRON- service
fast and good experience
good service good experience
-PRON- have be have -PRON- truck service here for over 10_year
-PRON- be always be a quick pleasant experience
no issue
-PRON- be a great experience -PRON- be very nice and courteous
-PRON- would definitely go there again
great experience excellent job
-PRON- have a great experience at the willow_grove valvoline
-PRON- be have an issue with the groupon code but be able to honor -PRON- groupon anyway
-PRON- be super quick and give -PRON- the price breakdown of everything from the beginning
-PRON- would definitely go there again
the experience be very nice
the staff be nice -PRON- would not mind go there again for -PRON- service
very pleasant experience
-PRON- would never go to the one in the mobile gas station such rude people
-PRON- get to sit in the car

look at the website to understand what -PRON- be about
no tip to share for year friend have suggest that i try feldenkrais as a way to help with migraine bad back neck pain etc
so -PRON- do and yes after one brief hour of very small movement -PRON- feel relief
-PRON- should listen to -PRON- friend more often
nice place and people
call first to see if session be full or if -PRON- want to know_what the particular teacher focus_on
very friendly and efficient desk staff
-PRON- use the term basic and beginner interchangeably -PRON- mean the same thing
what be not clear before -PRON- start be that each class be a different lesson
that mean that the instruction will vary from class to class
dress casually relax and enjoy
interesting approach to renew body movement
just relax and do less
arrive about 5_min early to class
the changing bathroom be waay in the back
great for performer or individual with muscle or joint strain
the open class really make -PRON- present and be a great -PRON- time aw

the massage make up for what -PRON- lack in aesthetic
the massage be ok
although -PRON- go in -PRON- shoulder_where fine but when -PRON- leave out -PRON- be both so sore and even the next_morning -PRON- be still_sore
the lady do quite a bit of talking_during the massage -PRON- would have prefer_quietness
in -PRON- also_force -PRON- to tip -PRON- a certain_amount and nothing_under
which -PRON- think be ridiculous a tip be a tip
staff be very friendly and the procedure be quick than expect
-PRON- be too_noisy
the massage be great but -PRON- could_hear people talk next_foor
-PRON- be a great massage
do a brazilian_wax during -PRON- morning_coffee break
-PRON- do -PRON- in maybe 10_15 minute and -PRON- be one shot and do
-PRON- be truly professional and do not leave_careless strand_or hair like other place
nothing bad than find a strip of hair later on at pro_skincare -PRON- have -PRON- vacation_ready
on_february 10th_2017 -PRON- be schedule for a full_set of mink_eyelash
the person_who ap

...

can not wait to come back
one of -PRON- favorite
too many tomato in the salad
-PRON- be a little overwhelming
otherwise perfect
some of the food be not hot enough
cozmo leave a last impression and be from la -PRON- will definitely recommend onegin to family and friend
no sugar in the crepe and serve -PRON- real blini
recommend
good for date_night
two_thumb up
-PRON- do have a great time at onegin
very long wait for your food no reason
-PRON- waiter make excellent recommendation
-PRON- really enjoy everything -PRON- order
-PRON- feel_like -PRON- be eat at -PRON- grandmother 's house from 50 year_ago
great vodka
-PRON- have such a great time
thank -PRON-
wish the groupon menu be more expand to sample more dish
-PRON- and -PRON- boyfriend absolutly love this place and be come back very soon
keep do -PRON- good job love -PRON-
-PRON- will be back
unfortunate that the have a groupon specific menu
wonderful place- can not wait to come back.already recommend to several friend
place be very ni

-PRON- not what i pay for.couple package include 60 $ value champagne or chocolate
-PRON- do not drink so -PRON- get 2 lindt chocolate and a few chocolate cover mini pretzel from the jar and 2 skinny cow ice coffee
that be not even $_10 value
to schedule appointment -PRON- take several attempt tell -PRON- to call back and nobody available
massage be ok but not great -PRON- keep get out of the room
-PRON- should have everything inside ready to go
great customer service
natalie be great
service be speedy in a good way
do not make a late evening appointment
otherwise service be excellent
ambience be a little dated but the service be good
the people be nice
great service totally recommend
the service be excellent as_well as the staff
-PRON- husband and -PRON- have a great spa day
this place be run_down dirty and unprofessional
-PRON- would rather get a massage at -PRON- nail_salon then here
-PRON- have an excellent treatment
the staff be so friendly and warm
-PRON- be always a treat for -P

if -PRON- get the keratin and -PRON- hair be curly have -PRON- use -PRON- 24_karat formula
-PRON- be the good
-PRON- have be to many other salon but their product last not just 1 month but for beyond 3_month
-PRON- will not be disappoint
leave -PRON- enough time
great experience love -PRON- keratin hair
take the subway find street parking be difficult and lot be expensive
everyone leave with fabulous hair
do a great job with -PRON- keratin
-PRON- hair look great place a little unorganized and not -PRON- typical salon but result be worth -PRON-
-PRON- daughter go to gina -PRON- use to take an_hour to straighten -PRON- hair this morning -PRON- wash -PRON- hair after leave the keratin in for 48_hour -PRON- hair be straight and beautiful thank -PRON- gina
make_sure u have an appt
be on time
nyc ladiesthis place be a gemits a hair bar that really transform -PRON- hairlovely lady serve with smilevery professional thank -PRON- tammy -PRON- make -PRON- dayand the coming month with lovely hair


no one should have to be expose to such occurance
-PRON- ne to give people a relaxing experience
-PRON- leave angry
great quality beef
-PRON- sell all kind of beef duck beal lamb
-PRON- now have the downstair open as the first floor
not to miss -PRON- bone broth
top quality product no question
customer service could improve
on two occasion -PRON- have stand at the front counter for several minute with few to no other customer and not one employee ask if -PRON- need assistance
-PRON- will probably seek other establishment that make -PRON- feel that -PRON- care for -PRON- customer
porterhouse every time
very high_quality merchandise very polite and helpful staff
-PRON- respect the animal and do not sell meat from factory farm
-PRON- egg be from pasteured chicken and -PRON- use only product from local farm
the people who work there be really nice and helpful
-PRON- be head to an outdoor event after -PRON- purchase and ask for double wrap to try to keep the meat cool for 3hrs
the counterpe

KeyboardInterrupt: 

In [28]:
trigram_model_filepath = os.path.join(os.getcwd(),'trigram_model_all')

In [29]:
%%time
trigram_model = Phrases(bigram_sentences)
trigram_model.save(trigram_model_filepath)
trigram_model = Phrases.load(trigram_model_filepath)

CPU times: user 3.62 s, sys: 72.6 ms, total: 3.69 s
Wall time: 3.7 s


In [30]:
trigram_sentences_filepath = os.path.join(os.getcwd(),'trigram_sentences_all.txt')

In [31]:
with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for bigram_sentence in bigram_sentences:
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
        f.write(trigram_sentence + '\n')

/Users/nicholasmaloof/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [32]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [ ]:
for trigram_sentence in trigram_sentences:
    print(' '.join(trigram_sentence))
    print('')

In [33]:
trigram_reviews_filepath = os.path.join(os.getcwd(),'trigram_transformed_reviews_all.txt')

In [34]:
%%time
additional_stop = ['good', 'great', 'like', 'love']
with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
    for parsed_review in nlp.pipe(line_review(review_txt_filepath), batch_size=10000, n_threads=3):
        unigram_review = [token.lemma_ for token in parsed_review if not punct_space(token)]
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]
        trigram_review = [term for term in trigram_review if term not in spacy.en.English.Defaults.stop_words]
        trigram_review = [term for term in trigram_review if term not in additional_stop]
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

/Users/nicholasmaloof/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1min 47s, sys: 342 ms, total: 1min 47s
Wall time: 1min 48s


In [ ]:
print('Transformed:' + u'\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in f:
        print(review)

In [35]:
trigram_dictionary_filepath = os.path.join(os.getcwd(), 'trigram_dict_all.dict')

In [36]:
%%time

trigram_reviews = LineSentence(trigram_reviews_filepath)
trigram_dictionary = Dictionary(trigram_reviews)
# trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
trigram_dictionary.compactify()
trigram_dictionary.save(trigram_dictionary_filepath)

trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 1.8 s, sys: 14.7 ms, total: 1.81 s
Wall time: 1.81 s


In [37]:
trigram_bow_filepath = os.path.join(os.getcwd(), 'trigram_bow_corpus_all.mm')

In [38]:
def trigram_bow_generator(filepath):
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [39]:
%%time

MmCorpus.serialize(trigram_bow_filepath, trigram_bow_generator(trigram_reviews_filepath))
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 2.76 s, sys: 71.4 ms, total: 2.83 s
Wall time: 2.86 s


In [42]:
lda_model_filepath = os.path.join(os.getcwd(), 'lda_model_all')

In [43]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda = LdaMulticore(trigram_bow_corpus, num_topics=3, id2word=trigram_dictionary, workers=3, passes=25)
lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)


CPU times: user 8min, sys: 52 s, total: 8min 52s
Wall time: 9min 7s


In [44]:
def explore_topic(topic_number, topn=25):
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [45]:
explore_topic(topic_number=0)

term                 frequency

service              0.088
food                 0.067
excellent            0.024
place                0.023
staff                0.020
nice                 0.020
friendly             0.018
delicious            0.013
fast                 0.012
quick                0.012
amazing              0.011
order                0.009
customer             0.008
atmosphere           0.008
drink                0.008
price                0.008
restaurant           0.007
deal                 0.007
view                 0.006
clean                0.006
awesome              0.005
efficient            0.005
come                 0.005
small                0.005
burger               0.005


In [46]:
LDAvis_data_filepath = os.path.join(os.getcwd(), 'ldavis_prepared')

In [47]:
%%time

LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, trigram_dictionary)

/Users/nicholasmaloof/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


CPU times: user 33.8 s, sys: 211 ms, total: 34 s
Wall time: 34.6 s


In [48]:
pyLDAvis.display(LDAvis_prepared)

In [ ]:
Groupons.to_csv('groupon_deals_fixed.csv')

'/Users/nicholasmaloof/CodingProjects/WebScrap'